In [ ]:
import os
import ast
import torch
import argparse
import evaluate
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from accelerate import Accelerator
from torch.utils.data import DataLoader
from seqeval.metrics import classification_report
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, get_scheduler, DataCollatorForTokenClassification

In [ ]:
# import requests

# # Replace with your actual API key

# https://www.opensanctions.org/entities/NK-jFf3vbLSJD5sviAomgsney/
# API_KEY = "your_api_key_here"
# url = "https://api.opensanctions.org/v1/persons/"  # Example endpoint

# headers = {
#     "Authorization": f"Bearer {API_KEY}",
#     "Content-Type": "application/json"
# }

# # Optional: add query parameters if needed
# params = {
#     "name": "John Doe"  # Example: Searching for a person by name
# }

# response = requests.get(url, headers=headers, params=params)

# if response.status_code == 200:
#     data = response.json()  # Get the JSON response
#     print(data)
# else:
#     print(f"Error: {response.status_code} - {response.text}")


In [ ]:
# Data generation

In [ ]:

!pip install datasets
!pip install transformers
!pip install accelerate -U
!pip install evaluate
!pip install seqeval
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=82d84ca128d4cf5f732739ab3978a7bf09a0cec04d35f4cf31d7e3432ae71403
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


Generating Negative News: The function generate_negative_news() creates random negative news examples based on predefined templates.
NER Extraction: The perform_ner() function processes each news article using spaCy’s NER capabilities and extracts named entities.

In [ ]:
!pip install spacy


In [ ]:
import random
# Function to generate negative news examples
def generate_negative_news():
    news_templates = [
        ("{organization} Faces {adjective} Lawsuit Over {incident}", ["company", "company", "incident"]),
        ("{person} Resigns Amid {scandal}", ["person", "scandal"]),
        ("{person} Faces Charges After {crime} Scheme", ["person", "crime"]),
        ("{organization} Under Investigation for {incident} After {event}", ["organization", "incident", "event"]),
        ("{organization} Faces {legal_issue} Over Discriminatory {practice}", ["organization", "legal_issue", "practice"]),
    ]

    organizations = ["XYZ Corp", "ABC Enterprises", "DEF Bank", "Global Tech Inc.", "ABC Airlines"]
    persons = ["John Smith", "Jane Doe", "Mike Johnson", "Sarah Connor"]
    incidents = ["data breach", "financial scandal", "safety violations", "corruption charges"]
    crimes = ["kickback", "fraudulent", "money laundering"]
    scandals = ["financial scandal", "illegal funding", "bribery"]
    legal_issues = ["class-action lawsuit", "legal scrutiny", "corruption investigation"]
    practices = ["lending practices", "hiring practices", "environmental standards"]
    adjectives = ["record-breaking", "massive", "unprecedented", "severe"]
    events = ["deadly crash", "collapse of stock", "public protest", "failing inspection"]

    # Randomly select words for the template
    template, entities = random.choice(news_templates)
    selected_entities = {
        "organization": random.choice(organizations),
        "person": random.choice(persons),
        "incident": random.choice(incidents),
        "crime": random.choice(crimes),
        "scandal": random.choice(scandals),
        "legal_issue": random.choice(legal_issues),
        "practice": random.choice(practices),
        "adjective": random.choice(adjectives),
        "event": random.choice(events),
    }

    # Fill the template with random words
    news = template.format(**selected_entities)

    return news

# Example usage:
for _ in range(5):  # Generate 5 negative news articles
    print(generate_negative_news())
    print("\n" + "-"*50 + "\n")

Global Tech Inc. Under Investigation for financial scandal After deadly crash

--------------------------------------------------

XYZ Corp Under Investigation for corruption charges After failing inspection

--------------------------------------------------

XYZ Corp Under Investigation for safety violations After public protest

--------------------------------------------------

XYZ Corp Faces severe Lawsuit Over corruption charges

--------------------------------------------------

ABC Airlines Faces legal scrutiny Over Discriminatory lending practices

--------------------------------------------------



In [ ]:
import spacy

# Load the pre-trained spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to perform NER
def perform_ner(text):
    doc = nlp(text)
    entities = [(entity.text, entity.label_) for entity in doc.ents]
    return entities

# Example usage:
for _ in range(5):  # Generate 5 negative news articles
    news_article = generate_negative_news()
    print(f"News Article: {news_article}")

    # Get entities from the news article
    entities = perform_ner(news_article)
    print("Named Entities:", entities)
    print("\n" + "-"*50 + "\n")


News Article: Sarah Connor Faces Charges After kickback Scheme
Named Entities: [('Sarah Connor Faces Charges', 'ORG')]

--------------------------------------------------

News Article: Sarah Connor Faces Charges After fraudulent Scheme
Named Entities: [('Sarah Connor Faces Charges', 'ORG')]

--------------------------------------------------

News Article: Global Tech Inc. Faces corruption investigation Over Discriminatory hiring practices
Named Entities: [('Global Tech Inc. Faces', 'ORG')]

--------------------------------------------------

News Article: DEF Bank Under Investigation for corruption charges After public protest
Named Entities: [('DEF Bank Under Investigation', 'ORG')]

--------------------------------------------------

News Article: DEF Bank Faces corruption investigation Over Discriminatory environmental standards
Named Entities: [('DEF Bank Faces', 'ORG')]

--------------------------------------------------



In [ ]:
!pip install transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# from transformers import LlamaForCausalLM, LlamaTokenizer
# import torch

# # Step 1: Load the model and tokenizer
# model_name = "unsloth/Meta-Llama-3.1-8B-Instruct"  # You can replace this with the correct model path if you're using a specific version of Llama
# tokenizer = LlamaTokenizer.from_pretrained(model_name)
# model = LlamaForCausalLM.from_pretrained(model_name)

# # Step 2: Define a function to generate negative news
# def generate_negative_news(prompt):
#     # Encode the prompt into tokens
#     inputs = tokenizer(prompt, return_tensors="pt")

#     # Generate a response from the model
#     with torch.no_grad():
#         outputs = model.generate(
#             **inputs,
#             max_length=250,  # Maximum length of the output (can adjust as needed)
#             num_return_sequences=1,  # Number of sequences to generate
#             no_repeat_ngram_size=2,  # Avoid repeating n-grams (to improve output diversity)
#             top_p=0.95,  # Sampling technique
#             top_k=50,  # Number of top candidates to sample from
#             temperature=0.7,  # Control randomness (lower is more deterministic)
#             pad_token_id=tokenizer.eos_token_id
#         )

#     # Decode the generated tokens into text
#     generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return generated_text

# # Step 3: Set up a prompt to generate negative news about organizations, individuals, or scandals
# prompt = (
#     "Generate a negative news article about a company or individual involved in a major scandal. "
#     "The company or individual should face accusations of corruption, fraud, or other illegal activity. "
#     "Include details like a lawsuit, CEO resignation, or criminal charges, and mention financial repercussions."
# )

# # Step 4: Generate and print the negative news article
# negative_news_article = generate_negative_news(prompt)
# print(negative_news_article)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import transformers
print(transformers.__version__)
print(torch.__version__)

def load_model(model_path):
    device = "cuda"
    tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
    model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16)
    return model, tokenizer
def prompt_model(model, tokenizer, prompt_user, prompt_system=None, verbose=True, max_new_token=128, temperature=0.5, top_p=0.9, do_sample=True):
    messages = []
    if prompt_system:
        messages.append({"role": "user", "content": prompt_system})
    messages.append({"role": "user", "content": prompt_user})
    input_text=tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(model.device)
    outputs = model.generate(inputs, max_new_tokens=max_new_token, temperature=temperature, top_p=top_p, do_sample=do_sample)
    response = tokenizer.decode(outputs[0])
    if verbose:
        print(response)
    return response

4.48.2
2.5.1+cu124


In [ ]:
# model, tokenizer = load_model("/h/ws_yuehuan_he/green-ai-explore/models/Llama-3.2-3b-Instruct")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# Step 1: Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-1.7B-Instruct"  # You can replace this with the other model paths from hugging face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,device_map = "auto")

# Step 2: Check if the model is loaded on GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Model is loaded on: {device}")
# Step 3: Define a function to generate negative news
def generate_negative_news(prompt):
    # Encode the prompt into tokens
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate a response from the model
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=250,  # Maximum length of the output (can adjust as needed)
            num_return_sequences=1,  # Number of sequences to generate
            no_repeat_ngram_size=2,  # Avoid repeating n-grams (to improve output diversity)
            top_p=0.95,  # Sampling technique
            top_k=50,  # Number of top candidates to sample from
            temperature=0.7,  # Control randomness (lower is more deterministic)
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the generated tokens into text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text
# Step 4: Set up a prompt to generate negative news about organizations, individuals, or scandals
prompt = (
    "Generate a negative news article about a company or individual involved in a major scandal. "
    "The company or individual should face accusations of corruption, fraud, or other illegal activity. "
    "Include details like a lawsuit, CEO resignation, or criminal charges, and mention financial repercussions."
)

# Step 5: Generate and print the negative news article
negative_news_article = generate_negative_news(prompt)
print(negative_news_article)



Model is loaded on: cuda


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Generate a negative news article about a company or individual involved in a major scandal. The company or individual should face accusations of corruption, fraud, or other illegal activity. Include details like a lawsuit, CEO resignation, or criminal charges, and mention financial repercussions.


In [ ]:

# Generate 2000 records in batches of 100
batch_size = 100  # Generate 100 records per batch
total_records = 2000
generated_records = []

# Loop to generate articles in batches
for i in range(total_records // batch_size):  # Adjust the range for batches
    batch = []
    for j in range(batch_size):
        article = generate_negative_news(prompt)
        batch.append(article)

    # Store the batch of generated articles
    generated_records.extend(batch)

    # Optionally print progress
    print(f"Generated batch {i+1}/{total_records // batch_size}.")

# Optionally: Save the generated records to a text file
with open("generated_negative_news.txt", "w") as f:
    for record in generated_records:
        f.write(record + "\n\n")

print(f"Generated {total_records} negative news articles.")

Generated batch 1/20.
Generated batch 2/20.
Generated batch 3/20.
Generated batch 4/20.
Generated batch 5/20.
Generated batch 6/20.
Generated batch 7/20.
Generated batch 8/20.
Generated batch 9/20.
Generated batch 10/20.
Generated batch 11/20.
Generated batch 12/20.
Generated batch 13/20.
Generated batch 14/20.
Generated batch 15/20.
Generated batch 16/20.
Generated batch 17/20.
Generated batch 18/20.
Generated batch 19/20.
